In [28]:
import requests
from dataset import get_recipe
from prompt import example_prompt
import pandas as pd
import json

In [2]:
url = 'http://127.0.0.1:8000/gemini/questions/en/'
url_desc = 'http://127.0.0.1:8000/gemini/description/en/'

url_quest = 'http://127.0.0.1:8000/gemini/answers/en/'

In [3]:
recipe = get_recipe(1)

In [4]:
recipe_name = recipe['name']
ingredients = recipe['ingredients']
steps = recipe['steps']
time = recipe['minutes']

In [35]:
steps

"['preheat oven to 425 degrees f', 'press dough into the bottom and sides of a 12 inch pizza pan', 'bake for 5 minutes until set but not browned', 'cut sausage into small pieces', 'whisk eggs and milk in a bowl until frothy', 'spoon sausage over baked crust and sprinkle with cheese', 'pour egg mixture slowly over sausage and cheese', 's& p to taste', 'bake 15-20 minutes or until eggs are set and crust is brown']"

In [6]:
data = {
    'recipe_name' : recipe_name,
    'ingredients' : ingredients,
    'steps' : steps,
    'time' : time
}

In [51]:
response = requests.post(url=url, data=data)

In [8]:
response.status_code

200

In [46]:
response.json()

{'response': '<step 1>\n1. Why is the oven preheated to 425 degrees F specifically?\n2. What type of pizza crust is recommended for this recipe?\n</step 1>\n<step 2>\n1. What is the purpose of pressing the dough into the bottom and sides of the pizza pan?\n2. What is the best method for pressing the dough into the pan?\n</step 2>\n<step 3>\n1. Why is the crust baked for only 5 minutes?\n2. How can you determine if the crust is set but not browned?\n</step 3>\n<step 4>\n1. What type of sausage is recommended for this recipe?\n2. How small should the sausage pieces be cut?\n</step 4>\n<step 5>\n1. What type of milk is recommended for this recipe?\n2. How much milk should be used per serving?\n3. What is the purpose of whisking the eggs and milk until frothy?\n</step 5>\n<step 6>\n1. What is the best method for spooning the sausage over the baked crust?\n2. What type of cheese is recommended for this recipe?\n3. How much cheese should be used per serving?\n</step 6>\n<step 7>\n1. What is 

In [10]:
desc = requests.post(url=url_desc, data=data)

In [28]:
desc.status_code

200

In [11]:
desc.json()

{'description': 'Indulge in the unexpected with our Breakfast Pizza, a morning delight that combines the savory flavors of sausage, melted cheese, and fluffy eggs atop a crispy pizza crust. This unique take on breakfast is sure to elevate your morning routine, offering a satisfying and delectable start to your day.'}

In [12]:
data_quest = {
    'recipe_name' : recipe_name,
    'ingredients' : ingredients,
    'steps' : steps,
    'time' : time,
    
    'step_num' : 2,
    'question' : 'How much dough is needed per serving?'
}

In [13]:
from prompt import list_to_str
list_to_str(steps)

'1. preheat oven to 425 degrees f\n2. press dough into the bottom and sides of a 12 inch pizza pan\n3. bake for 5 minutes until set but not browned\n4. cut sausage into small pieces\n5. whisk eggs and milk in a bowl until frothy\n6. spoon sausage over baked crust and sprinkle with cheese\n7. pour egg mixture slowly over sausage and cheese\n8. s& p to taste\n9. bake 15-20 minutes or until eggs are set and crust is brown'

In [60]:
steps

"['preheat oven to 425 degrees f', 'press dough into the bottom and sides of a 12 inch pizza pan', 'bake for 5 minutes until set but not browned', 'cut sausage into small pieces', 'whisk eggs and milk in a bowl until frothy', 'spoon sausage over baked crust and sprinkle with cheese', 'pour egg mixture slowly over sausage and cheese', 's& p to taste', 'bake 15-20 minutes or until eggs are set and crust is brown']"

In [37]:
ans = requests.post(url=url_quest, data=data_quest)

In [63]:
ans.status_code

200

In [66]:
ans.reason

'OK'

In [38]:
ans.json()

{'user': {'role': 'user',
  'parts': 'Question for step 2: How much dough is needed per serving?'},
 'model': {'role': 'model',
  'parts': 'This recipe uses one prepared pizza crust for a 12-inch pizza pan, which generally translates to about 6-8 servings depending on how you slice it. \n'},
 'answer': 'This recipe uses one prepared pizza crust for a 12-inch pizza pan, which generally translates to about 6-8 servings depending on how you slice it. \n'}

In [39]:
hist = [ans.json()['user'], ans.json()['model']]

In [40]:
data_quest2 = {
    'recipe_name' : recipe_name,
    'ingredients' : ingredients,
    'steps' : steps,
    'time' : time,
    
    'step_num' : 3,
    'question' : 'How long should the dough be baked for?',
    'history' : json.dumps(hist),
}

In [41]:
from pprint import pprint
pprint(data_quest2)

{'history': '[{"role": "user", "parts": "Question for step 2: How much dough '
            'is needed per serving?"}, {"role": "model", "parts": "This recipe '
            'uses one prepared pizza crust for a 12-inch pizza pan, which '
            'generally translates to about 6-8 servings depending on how you '
            'slice it. \\n"}]',
 'ingredients': "['prepared pizza crust', 'sausage patty', 'eggs', 'milk', "
                "'salt and pepper', 'cheese']",
 'question': 'How long should the dough be baked for?',
 'recipe_name': 'a bit different  breakfast pizza',
 'step_num': 3,
 'steps': "['preheat oven to 425 degrees f', 'press dough into the bottom and "
          "sides of a 12 inch pizza pan', 'bake for 5 minutes until set but "
          "not browned', 'cut sausage into small pieces', 'whisk eggs and milk "
          "in a bowl until frothy', 'spoon sausage over baked crust and "
          "sprinkle with cheese', 'pour egg mixture slowly over sausage and "
          "ch

In [42]:
ans2 = requests.post(url=url_quest, data=data_quest2)

In [76]:
ans2.status_code

401

In [43]:
ans2.text

'{"user": {"role": "user", "parts": "Question for step 3: How long should the dough be baked for?"}, "model": {"role": "model", "parts": "Step 3 says to bake the crust for 5 minutes, just until it\'s set but not browned. This is called \\"pre-baking\\" or \\"blind baking\\" the crust, and it helps prevent a soggy bottom.  \\n"}, "answer": "Step 3 says to bake the crust for 5 minutes, just until it\'s set but not browned. This is called \\"pre-baking\\" or \\"blind baking\\" the crust, and it helps prevent a soggy bottom.  \\n"}'

In [44]:
ans2.json()

{'user': {'role': 'user',
  'parts': 'Question for step 3: How long should the dough be baked for?'},
 'model': {'role': 'model',
  'parts': 'Step 3 says to bake the crust for 5 minutes, just until it\'s set but not browned. This is called "pre-baking" or "blind baking" the crust, and it helps prevent a soggy bottom.  \n'},
 'answer': 'Step 3 says to bake the crust for 5 minutes, just until it\'s set but not browned. This is called "pre-baking" or "blind baking" the crust, and it helps prevent a soggy bottom.  \n'}